# Query & search registries

Find & access data using registries.

## Setup

In [ ]:
!lamin init --storage ./mydata

In [ ]:
import lamindb as ln

ln.settings.verbosity = "info"

We'll need some toy data:

In [ ]:
ln.Artifact(ln.core.datasets.file_jpg_paradisi05(), description="My image").save()
ln.Artifact.from_df(ln.core.datasets.df_iris(), description="The iris collection").save()
ln.Artifact(ln.core.datasets.file_fastq(), description="My fastq").save()

## Look up metadata

For entities where we don't store more than 100k records, a look up object can be a convenient way of selecting a record.

Consider the `User` registry:

In [ ]:
users = ln.User.lookup(field="handle")

With auto-complete, we find a user:

In [ ]:
user = users.testuser1

In [ ]:
user

:::{note}

You can also auto-complete in a dictionary:

```python
users_dict = ln.User.lookup().dict()
```

:::

## Filter by metadata

Filter for all artifacts created by a user:

In [ ]:
ln.Artifact.filter(created_by=user).df()

To access the results encoded in a filter statement, execute its return value with one of:

- `.df()`: A pandas `DataFrame` with each record stored as a row.
- `.all()`: An indexable django `QuerySet`.
- `.list()`: A list of records.
- `.one()`: Exactly one record. Will raise an error if there is none.
- `.one_or_none()`: Either one record or `None` if there is no query result.

```{note}

{meth}`~lamindb.core.Registry.filter` returns a {class}`~lamindb.core.QuerySet`.

The ORMs in LaminDB are Django Models and any [Django query](https://docs.djangoproject.com/en/stable/topics/db/queries/) works. LaminDB extends Django's API for data scientists.

Under the hood, any `.filter()` call translates into a SQL select statement.

`.one()` and `.one_or_none()` are two parts of LaminDB's API that are borrowed from SQLAlchemy.

```

## Search for metadata

In [ ]:
ln.Artifact.search("iris").df()

Let us create 500 notebook objects with fake titles and save them:

In [ ]:
ln.save(
    [
        ln.Transform(name=title, type="notebook")
        for title in ln.core.datasets.fake_bio_notebook_titles(n=500)
    ]
)

We can now search for any combination of terms:

In [ ]:
ln.Transform.search("intestine").df().head()

## Leverage relations

Django has a double-under-score syntax to filter based on related tables.

This syntax enables you to traverse several layers of relations:

In [ ]:
ln.Artifact.filter(run__created_by__handle__startswith="testuse").df()

The filter selects all artifacts based on the users who ran the generating notebook.

(Under the hood, in the SQL database, it's joining the artifact table with the run and the user table.)



Beyond `__startswith`, Django supports about [two dozen field comparators](https://docs.djangoproject.com/en/stable/ref/models/querysets/#field-lookups) `field__comparator=value`.

Here are some of them.

### and

In [ ]:
ln.Artifact.filter(suffix=".jpg", created_by=user).df()

### less than/ greater than

Or subset to artifacts greater than 10kB. Here, we can't use keyword arguments, but need an explicit where statement.

In [ ]:
ln.Artifact.filter(created_by=user, size__lt=1e4).df()

### or

In [ ]:
from django.db.models import Q

ln.Artifact.filter().filter(Q(suffix=".jpg") | Q(suffix=".fastq.gz")).df()

### in

In [ ]:
ln.Artifact.filter(suffix__in=[".jpg", ".fastq.gz"]).df()

### order by

In [ ]:
ln.Artifact.filter().order_by("-updated_at").df()

### contains

In [ ]:
ln.Transform.filter(name__contains="search").df().head(10)

And case-insensitive:

In [ ]:
ln.Transform.filter(name__icontains="Search").df().head(10)

### startswith

In [ ]:
ln.Transform.filter(name__startswith="Research").df()

In [ ]:
# clean up test instance
!lamin delete --force mydata
!rm -r mydata